In [28]:
import whisper
import ffmpeg
import numpy as np
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model = whisper.load_model("base").to(device)

In [40]:
# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("filename")
audio = whisper.pad_or_trim(audio)
# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# decode the audio
options = whisper.DecodingOptions(fp16 = False, language = 'slovenian')
result = whisper.decode(model, mel, options)

# print the recognized text
print(result.text)

Detected language: hr
Pogovalimo se obobri. Zdaj.


In [39]:
import pyaudio
import wave

if __name__ == "__main__":
    DEVICE_INDEX = 10
    
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 48000
    CHUNK = 1024
    RECORD_SECONDS = 6
    WAVE_OUTPUT_FILENAME = 'filename'

    audio = pyaudio.PyAudio()

    #pa = pyaudio.PyAudio()
    #list_audio = []
    #for x in range(0, pa.get_device_count()):
    #    info = pa.get_device_info_by_index(x)
    #    print("info {0}".format(info))

    # start Recording
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK,
                        input_device_index=DEVICE_INDEX)
                        # sample_rate=RATE)
    print("recording...")
    print('---------------------------------')
    print(int(RATE / CHUNK * RECORD_SECONDS))
    print('*********************************')

    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        print("Recording . . .")

        frames.append(data)
    print("Recording finished. . .")

    # stop Recording
    stream.stop_stream()
    stream.close()
    audio.terminate()

    waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    waveFile.setnchannels(CHANNELS)
    waveFile.setsampwidth(2)
    audio.get_sample_size(FORMAT)
    waveFile.setframerate(RATE)
    waveFile.writeframes(b''.join(frames))
    waveFile.close()

recording...
---------------------------------
281
*********************************
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Recording . . .
Rec